In [ ]:
import nltk # natural language processing
import csv # needed to load csv files
import numpy as np
import pandas as pd # for creation of dataframes later
import pickle # for saving lists
import re
import random # needed to randomly select characters from string
from collections import Counter # to count vocabulary

arr_of_langs = ["Slovak","French","Spanish","German","Polish"]

In [ ]:
# define function to load csv files and remove headers
def LoadCSV(filename):
    raw_file = open(filename, encoding="utf8")

    # determine if the file has a header
    has_header = csv.Sniffer().has_header(raw_file.read(1024))
    raw_file.seek(0)
    
    # read the csv and make the data a list
    incsv = csv.reader(raw_file)
    data = list(incsv)
    
    # remove the first row if there is a header
    if has_header:
        data = data[1:]
    
    return data

# load the training set x
x_filename = '../data/train_set_x.csv'
x_values = LoadCSV(x_filename)
print(len(x_values))
print(x_values[18])

# And the training set y
y_filename = '../data/train_set_y.csv'
y_values = LoadCSV(y_filename)
print(len(y_values))
print(y_values[18])

In [ ]:
# add a vector of the classes (y_values) to the data (x_values)

just_class = [x[1] for x in y_values]

def append(dataset, classes):
    for i in range(len(dataset)):
        dataset[i].append(classes[i])
    return dataset

full_data = append(x_values, just_class)
just_text = [x[1] for x in full_data]
print(full_data[20:25])
print(just_text[20:25])
type(full_data)

In [ ]:
# going to make the data comparable to the test set by sampling 2 letters from each word
# and creating a just_letters list that we can then produce a vector space model for to use in a decision-tree

list_of_characters = []
number_of_words = []
random_characters = []

def convert_to_letters(dataset):
    for sentence in dataset:
        characters = []
        for c in sentence.lower():
            if not c == ' ':
                characters.append(c)
        
        # add random characters drawn from each string. we draw 2 times the number of words in the sentence.
        random_characters.append(random.choices(characters, k=(len(sentence.split())*2)))
    
    return random_characters

character_data = convert_to_letters(just_text)
print(character_data[:10])

In [ ]:
# need to create a listing of all the characters that appear - we will call this character_list
type(character_data)

character_list = list(set([element for tupl in character_data for element in tupl]))
len(character_list)

In [ ]:
# Create a vector space model with character_list parameters (424) for the training set - this takes some time
doc_term_matrix = []

def tf(term, document):
  return freq(term, document)

def freq(term, document):
  return document.count(term)

for count, doc in enumerate(character_data):
    tf_vector = [tf(character, doc) for character in character_list]
    tf_vector_string = ', '.join(format(freq, 'd') for freq in tf_vector)
    doc_term_matrix.append(tf_vector)
    
    if count % 10000 == 0:
        print(count)   
        
print(doc_term_matrix[:10])
len(doc_term_matrix)

In [ ]:
# THE FOLLOWING IS DERIVED FROM THE FOLLOWING GUIDES
# http://kldavenport.com/pure-python-decision-trees/
# https://machinelearningmastery.com/implement-decision-tree-algorithm-scratch-python/

# BUT MOSTLY FROM THE BASE CODE FROM: 
# https://github.com/random-forests/tutorials/blob/master/decision_tree.ipynb

# I have annotated the code clearly and adjusted where appropriate to more accurately meet the needs of the assignment, 
# but the skeleton of the core code (Questions, gini, info_gain, partition, find_best_split, and build_tree functions)
# is drawn from random-forests. I have cited them in the report.

vector_space = doc_term_matrix
header = character_list

print(len(vector_space)) # should be ~270000
print(len(just_class)) # should be ~270000
print(len(header)) # should be 424


In [ ]:
# IMPORT THE OUTPUTS FROM THE INITIAL vector_space PYTHON NOTEBOOK
# These are the vector_space, the classes of the training data, and the full character list being evaluated upon

# import pickle

# open_name = '../data/doc_term_matrix.txt'

# count = 0

# we created our vector space model and training classes in file "vector_space"

# first load up the data
# vector_space = pickle.load(open(open_name, "rb"))   
# just_class = pickle.load(open("../data/just_class.txt", "rb"))
# header = pickle.load(open('../data/character_list.txt', "rb"))
# len(just_class)

In [ ]:
# APPEND THE DATA AND CLASS TOGETHER FOR OUR TREE

def append(dataset, classes):
    for i in range(len(dataset)):
        dataset[i].append(classes[i])
    return dataset

full_data = append(vector_space, just_class)

# show all the different characters I will use for the vector space model
print(header)
print(len(full_data))
print(full_data[:1])

In [ ]:
# FUNCTION TO DETERMINE THE SIZE OF EACH CLASS

def class_counts(rows):
    counts = {}  # a dictionary that will contain label and count
    for row in rows:
        # the class is the last column in our data
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

# this shows the number for each class
class_counts(full_data)

In [ ]:
# CLASS QUESTION TO PARTITION THE DATASET BY A SPECIFIC QUESTION

class Question:
    
    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        return val >= self.value

In [ ]:
# CALCULATE THE UNCERTAINTY (GINI) IN THE DATA

def gini(rows):

    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

current_uncertainty = gini(full_data)

In [ ]:
# FUNCTION TO CALCULATE THE INFORMATION GAIN FROM A SPECIFIC QUESTION

def info_gain(left, right, current_uncertainty):
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [ ]:
# FUNCTION TO PARTITION A DATASET. BASED ON A SPECIFIC QUESTION BREAKS INTO THE BRANCHES

def partition(rows, question):
    
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

true_rows, false_rows = partition(full_data, Question(10, 1))
info_gain(true_rows, false_rows, current_uncertainty)

In [ ]:
# FIND THE BEST QUESTION TO ASK BY CHECKING EVERY FEATURE-VALUE PAIR AND CALCULATING INFORMATION GAIN

def find_best_split(rows):

    best_gain = 0  # start empty but will be filled by the question that does the best
    best_question = None  # track the question
    current_uncertainty = gini(rows) # get our uncertainty value (in case it is not already calculated)
    n_features = len(rows[0]) - 1  # number of columns in the data (-1 because of the value)

    for col in range(n_features):  # for each column/feature

        values = set([row[col] for row in rows])  # get the unique values

        for val in values:  # for each unique value

            question = Question(col, val) # confirm the question that will be the most useful

            # try splitting the dataset
            # if it does not divide the dataset (i.e. all true or all false, then skip it)
            true_rows, false_rows = partition(rows, question) 
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # If this gain is the best we have found, then save it 
            if gain > best_gain:
                best_gain = gain
                best_question = question

    return best_gain, best_question

In [ ]:
# ADD A NEW CLASS LEAF - HOLDS THE RELATIONSHIP BETWEEN A CLASS AND A LEAF

class Leaf:
    def __init__(self, rows):
        self.predictions = class_counts(rows)
        
# ADD A NEW DECISION NODE CLASS. Find the question and the two associated branches.

class Decision_Node:

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [ ]:
# FINALLY, WE CAN BUILD THE TREE

# this is a recursive tree that only stops when it finds no new information gain...we want to limit the number of max_features

def build_tree(rows, count):
    
    count = count + 1
    # First we find the best question that we can ask
    gain, question = find_best_split(rows)

    # When we do not gain any more information, this is a leaf and ends that line of inquiry
    # could tweak this parameter to only get things with better gains...this is an the intensive task, however.
    if gain == 0:
        return Leaf(rows)
    
    # this also ends if we are at the fifth iteration of finding true or false branches
    if count == 5:
        print("count is at 5 - creating a leaf...")
        count = 0
        return Leaf(rows)

    # Assuming we do not have a leaf, then there is a useful feature of the data that we can partition on
    true_rows, false_rows = partition(rows, question)

    # Start the recursion - build a branch for the trues
    true_branch = build_tree(true_rows, count)

    # and for the falses...
    false_branch = build_tree(false_rows, count)

    # Return a Question node. We now have our best feature/value to ask.
    # Note that this may overfit the data as it always looks for more information gain...
    # I should add something to limit it...
    return Decision_Node(question, true_branch, false_branch)

In [ ]:
import datetime
time = datetime.datetime.now()

# BUILD THE TREE - THIS TAKES A LONG TIME WITH THE FULL DATA AND DOES NOT
# SIGNIFICANTLY INCREASE PERFORMANCE. I RECOMMEND CHANGING TO USING ONLY A PORTION OF THE TRAINING SET
# OR A CROSS-VALIDATION TECHNIQUE

# decision_tree_10 = build_tree(full_data[:10], 0)
time =  datetime.datetime.now()-time
print(time)
time = datetime.datetime.now()

# decision_tree_100 = build_tree(full_data[:100],0)
time = datetime.datetime.now()-time
print(time)
time = datetime.datetime.now()

# decision_tree_1000 = build_tree(full_data[:1000],0)
time = datetime.datetime.now()-time
print(time)
time = datetime.datetime.now()

# decision_tree_10000 = build_tree(full_data[:10000],0)
time = datetime.datetime.now()-time
print(time)
time = datetime.datetime.now()

# decision_tree_100000 = build_tree(full_data[:100000],0)
time = datetime.datetime.now()-time
print(time)
time = datetime.datetime.now()

decision_tree_full = build_tree(full_data,0)
time = datetime.datetime.now()-time
print(time)
time = datetime.datetime.now()

In [ ]:
# prediction method

def classify(row, node):

    # If we have reached a leaf then we use that to determine the class
    if isinstance(node, Leaf):
        return node.predictions

    # If no leaf, then decide whether to follow the true-branch or the false-branch to recursively classify the data
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

# CONFRIM THAT CLASSIFY IS WORKING
list(classify(full_data[1], decision_tree_10000))[0]

In [ ]:
# CHECK THE ACCURACY AGAINST ~5% OF THE ORIGINAL DATA
prediction_training = []

for row in vector_space[263517:]:
    prediction_training.append(list(classify(row, decision_tree_full))[0])
    
prediction_verify = list(zip(prediction_training, just_class[263517:]))

# this gives the accuracy of our model on the training data...note that it is very low...
print(len([i for i, j in prediction_verify if i == j])/len(just_class[263517:]))

In [ ]:
# TIME TO PRODUCE THE OUTPUT.CSV file
# FIRST, LOAD THE TEST DATA
import csv

# load the test set x
x_filename = '../data/test_set_x.csv'
x_test = LoadCSV(x_filename)
print(len(x_test))
print(x_test[400])

In [ ]:
# REMOVE THE ID FROM THE TEST DATA
just_text = [x[1] for x in x_test]
just_text[:10]

In [ ]:
# CREATE A VECTOR SPACE MODEL FOR THE TEST DATA
doc_term_matrix = []

for count, doc in enumerate(just_text):
    tf_vector = [tf(character, doc) for character in header]
    tf_vector_string = ', '.join(format(freq, 'd') for freq in tf_vector)
    doc_term_matrix.append(tf_vector)
    
    if count % 10000 == 0:
        print(count)   
        
print(doc_term_matrix[:1])
len(doc_term_matrix)

In [ ]:
# MAKE PREDICTIONS FOR EACH ROW IN THE doc_term_matrix

model_used = decision_tree_full
model_name = 'decision_tree_full'

prediction = []

for row in doc_term_matrix:
    prediction.append(list(classify(row, model_used))[0])
    
prediction[:10]

In [ ]:
# APPEND THE ID TO THE PREDICTION VALUES
prediction_id = list(range(0,len(just_text)))
prediction_output = list(zip(prediction_id, prediction))

prediction_output[:10]

In [ ]:
# PRODUCE THE CSV IN REQUIRED OUTPUT FORMAT

model_string = '../submissions/output_scratch_%s.csv' %model_name

with open(model_string, "w", newline='') as csvfile:
    csv_out=csv.writer(csvfile)
    csv_out.writerow(['Id','Category'])
    for row in prediction_output:
        csv_out.writerow(row)